<a href="https://colab.research.google.com/github/dantheman625/nlp_doc_info_extraction/blob/complete_pipe/complete_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install seqeval scikit-learn

In [ ]:
print("hello world")

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    AutoModelForSequenceClassification,
    LongformerTokenizerFast,
    pipeline
)

## Wandb login

In [ ]:
import wandb
wandb.login()

# Datasets

Import Challenge data set (Final_eval.json)


## Mount Drive

In [ ]:
from google.colab import drive   # only in Colab; skip if you’re on a different setup
import os
import json

drive.mount('/content/drive', force_remount=True)

## Set Project folder

In [ ]:
drive_folder = "NLP_project_data"

## Load file

In [ ]:
base_path   = os.path.join('/content/drive/MyDrive', drive_folder)
eval_path   = os.path.join(base_path, 'Final_eval.json')

with open(eval_path, 'r', encoding='utf-8') as f:
    eval_data = json.load(f)

print(eval_data)

# Define models

# Baseline models
Define which model you used as a baseline model for the specific task

In [ ]:
baseline_ner_name = "allenai/longformer-base-4096"
baseline_re_name = "SpanBERT/spanbert-large-cased"


# Trained models

Define your trained model for the specific task

In [ ]:
trained_ner_name = "/path/to/your/ner/checkpoint"
trained_re_name = "/path/to/your/re/checkpoint"

# Model selection

Which model for NER, which for RE? -> Combination untrained/ untrained, trained/ trained, untrained/ trained, trained/ untrained

## Both baseline

In [ ]:
ner_model_name = baseline_ner_name
re_model_name  = baseline_re_name

## Both trained

In [ ]:
ner_model_name = trained_ner_name
re_model_name  = trained_re_name

## NER: trained, RE: baseline

In [ ]:
ner_model_name = trained_ner_name
re_model_name  = baseline_re_name

## NER: baseline, RE: trained

In [ ]:
ner_model_name = baseline_ner_name
re_model_name  = trained_re_name

# Load Models and Tokenizer

## NER

In [ ]:
## Update once Daniel has finished setup
ner_tokenizer = LongformerTokenizerFast.from_pretrained(re_model_name)
ner_model     = AutoModelForTokenClassification.from_pretrained(ner_model_name)

## RE

In [ ]:
re_tokenizer  = AutoTokenizer.from_pretrained(ner_model_name)
re_model      = AutoModelForSequenceClassification.from_pretrained(re_model_name)

#Initialize Wandb

In [ ]:
wandb.init(
    project="model-eval",
    name=f"eval_{ner_model_name.split('/')[-1]}_{re_model_name.split('/')[-1]}",
    config={
        "ner_model": ner_model_name,
        "re_model": re_model_name,
        "dataset": "Final_eval.json",
        "batch_size": 32,
        "max_length": 256,
        "seed": 42,
    }
)

# NER Eval

Output: Entity file -> content

In [ ]:
# placeholder
true_ner_labels = [["O", "O"]]
pred_ner_labels = [["O", "O"]]

## Log Metrics in Wandb

In [ ]:
from seqeval.metrics import precision_score as ner_prec, recall_score as ner_rec, f1_score as ner_f1
prec_ner = ner_prec(true_ner_labels, pred_ner_labels)
rec_ner  = ner_rec(true_ner_labels, pred_ner_labels)
f1_ner   = ner_f1(true_ner_labels, pred_ner_labels)

wandb.log({
    "ner/precision": prec_ner,
    "ner/recall":    rec_ner,
    "ner/f1":        f1_ner,
})


# RE Eval

Input: Entity file, original challenge test file -> matching of entities to sentences (siehe wa) -> Liste mit dict

## Extract unique label values for matching to docred

In [ ]:
unique_labels = set()
for item in eval_data:
    unique_labels.update(item.get("label_set", []))

# If you want them sorted for readability:
unique_labels = sorted(unique_labels)

print(unique_labels)


## Log metrics in Wandb

In [ ]:
# placeholder
true_re_labels = ["O", "O"]
pred_re_labels = ["O", "O"]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

prec_re, rec_re, f1_re, _ = precision_recall_fscore_support(
    true_re_labels, pred_re_labels, average="micro"
)

wandb.log({
    "re/precision": prec_re,
    "re/recall":    rec_re,
    "re/f1":        f1_re,
})


summary_table = wandb.Table(
    columns=[
      "ner_precision",
      "ner_recall",
      "ner_f1",
      "re_precision",
      "re_recall",
      "re_f1"
    ],
    data=[[prec_ner, rec_ner, f1_ner, prec_re, rec_re, f1_re]]
)
wandb.log({"metrics_summary": summary_table})

Wrap Up


In [ ]:
wandb.finish()